### Cargo packages e inicializo

In [2]:
import sys
sys.path.insert(1, '../')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import fcnAFIL
import lesion_classifiers as lc
import time
import SimpleITK as sitk

if os.environ['COMPUTERNAME']=='DIEGO-DESKTOP':
    datadir = 'E:/Entelai/DATA/threshold_finetuning/data completa'
    codedir = 'E:/REPOS/entelai_threshold_finetuning/'
else:
    datadir = 'C:/Entelai/DATA/threshold_finetuning/data completa'
    codedir = 'C:/REPOS/entelai_threshold_finetuning/'
    
outdatadir = os.path.join(datadir, 'out')


### Full process, file generation

In [ ]:

PAR = fcnAFIL.build_pairs_theshold_finetuning(datadir)

param = {"thres":np.nan, "slope":np.nan, "growtype":"largesmall_01p", \
    'fcn_classify_lesions':lc._classify_lesions_6SD,   \
        'condname':'largesmall_01p_new10'}

t = time.time()
for i, par in enumerate(PAR):
    print('Inicio %s'%(par['studiesname']))

    if len(par['studies'][0]['lesiones_fname'])==0:
        print('Empty filename')
        continue
   
    par = fcnAFIL.full_process_pair(par=par, outdatadir=outdatadir, param=param)

    # Delete some memory-consuming keys, to fit in memory
    if 'index' in par:
        del par['index']    
    for ind, study in enumerate(par['studies']):
        for keyremove in ['img', 'array', 'labels', 'volumen',
                            'lblchanges', 'relabeled']: # for back compatibility
            if keyremove in par['studies'][ind]:
                del par['studies'][ind][keyremove]

    PAR[i] = par

    t1 = time.time() - t
    # print(par)
    print("     Fin %s %2.2fs %s" % (par['studiesname'], t1, param['condname']))

np.savez_compressed(os.path.join(datadir,'PARES_threshold_finetuning.npz'), PAR=PAR)


### Cargo file generado

In [4]:
loaded = np.load(os.path.join(datadir,'PARES_threshold_finetuning.npz'), allow_pickle=True)
loaded.files
PAR = list(loaded['PAR'])

### Genero planilla unificada 

In [19]:
subject = []
id = []
v1 = []
v2 = []
for par in PAR:
    if not('LVTM' in par):
        continue
    lvtm = par['LVTM']
    for row in lvtm:
        subject.append(par['studiesname'])
        id.append(row[0])
        v1.append(row[1])
        v2.append(row[2])

df = pd.DataFrame(list(zip(subject,id,v1,v2)), columns=['subject','id','v1','v2'])

fcns = [lc._classify_lesions_4tau_samereso, lc._classify_lesions_6tau_samereso ,lc._classify_lesions_6tau_diffreso]
fcnnames = ['cat 4tau','cat 6tau samereso','cat 6tau diffreso']
for fcn,name in zip(fcns,fcnnames):
    index = fcn(df.v1,df.v2)
    categ = index['grow']*1+index['stable']*2+index['new']*3
    categs = ('error', 'grow', 'stable', 'new')    
    for i in range(len(categ)):
        categ[i]=categs[categ[i]]

    df[name] = categ
df.to_excel(os.path.join(outdatadir,'dataset_threshold_finetuning_all_lesions.xlsx'),index=False)
print('Listo')

Listo
